<a href="https://colab.research.google.com/github/IMOKURI/chaii-Hindi-and-Tamil-QA/blob/main/chaii_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📔 About this notebook ...

- 目的
    - postprocess の改善案模索
        - oof の結果と ground truth を比較する

# Prepare for Colab

In [1]:
import os
import sys
import zipfile

if os.path.exists('init.txt'):
    print("Already initialized.")

else:
    if 'google.colab' in sys.modules:
        from google.colab import drive
        drive.mount('/content/drive')
        dataset_dir = "/content/drive/MyDrive/Datasets"

        # ====================================================
        # Competition datasets
        # ====================================================
        with zipfile.ZipFile(f"{dataset_dir}/chaii-hindi-and-tamil-question-answering.zip", "r") as zp:
            zp.extractall(path="./")
        # with zipfile.ZipFile(f"{dataset_dir}/chaii-external-data-mlqa-xquad-preprocessing.zip", "r") as zp:
        #     zp.extractall(path="./")
        # with zipfile.ZipFile(f"{dataset_dir}/chaii-Squad_Translated_to_Tamil.zip", "r") as zp:
        #     zp.extractall(path="./")

    # for StratifiedGroupKFold
    # !pip uninstall -y scikit-learn
    # !pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn

    # for MultilabelStratifiedKFold
    # !pip install -q iterative-stratification

    # for CosineAnnealingWarmupRestarts
    # !pip install -qU 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'

    !pip install -q wandb
    # !pip install -q optuna

    # ====================================================
    # Competition specific libraries
    # ====================================================
    !pip install -q transformers
    !pip install -q sentencepiece
    # !pip install -q textstat
    # !pip install -q nlpaug

    !touch init.txt


Mounted at /content/drive
     |████████████████████████████████| 1.7 MB 4.1 MB/s 
     |████████████████████████████████| 133 kB 56.2 MB/s 
     |████████████████████████████████| 170 kB 60.2 MB/s 
     |████████████████████████████████| 97 kB 5.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 2.8 MB 4.3 MB/s 
     |████████████████████████████████| 636 kB 44.6 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 3.3 MB 45.5 MB/s 
     |████████████████████████████████| 895 kB 45.6 MB/s 
     |████████████████████████████████| 1.2 MB 4.2 MB/s 


# Library

In [2]:
# General libraries
import collections
import glob
import json
import math
import os
import random
import re
import statistics
import time
import warnings
from contextlib import contextmanager

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import torch
import torch.cuda.amp as amp
import torch.nn as nn
import torch.nn.functional as F
import wandb
# from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import mean_squared_error, jaccard_score
from sklearn.model_selection import KFold, StratifiedKFold  # , StratifiedGroupKFold
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm


In [3]:
# Competition specific libraries
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import nltk
# import textstat
import transformers as T


In [4]:
warnings.filterwarnings("ignore")


In [5]:
netrc = "/content/drive/MyDrive/.netrc" if 'google.colab' in sys.modules else "../input/wandbtoken/.netrc"
!cp -f {netrc} ~/
!wandb login

wandb_tags = []


wandb: Currently logged in as: imokuri (use `wandb login --relogin` to force relogin)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    wandb_tags.append(torch.cuda.get_device_name(0))


# Load Data

In [7]:
DATA_DIR = "./" if 'google.colab' in sys.modules else "../input/chaii-hindi-and-tamil-question-answering/"
OUTPUT_DIR = "./"
MODEL_DIR = "./models/"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)


In [8]:
train = pd.read_csv(DATA_DIR + "train.csv")
test = pd.read_csv(DATA_DIR + "test.csv")
sub = pd.read_csv(DATA_DIR + "sample_submission.csv")

#external_squad_translated_tamil = pd.read_csv(DATA_DIR + "squad_translated_tamil.csv")
#external_mlqa = pd.read_csv(DATA_DIR + "mlqa_hindi.csv")
#external_xquad = pd.read_csv(DATA_DIR + "xquad.csv")


# Config

In [9]:
# seed = random.randrange(10000)
seed = 440
print(seed)


440


In [10]:
class Config:
    wandb_entity = "imokuri"
    wandb_project = "chaii"
    print_freq = 100

    preprocess = False
    train = True
    validate = False
    inference = False

    debug = False
    num_debug_data = 50

    amp = False


In [11]:
config_defaults = {
    "seed": seed,
    # "n_class": 1,
    "n_fold": 5,
    "epochs": 2,
    "batch_size": 4,
    "gradient_accumulation_steps": 5,
    "max_grad_norm": 1.0,
    "criterion": "ChaiiCrossEntropyLoss",
    "optimizer": "BertAdamW",
    "scheduler": "get_cosine_schedule_with_warmup",
    "max_lr": 5e-5,
    "lr": 2e-5,
    "min_lr": 1e-5,
    "weight_decay": 0.01,
    "model_name": "deepset/xlm-roberta-large-squad2",
    # "model_name": "deepset/xlm-roberta-base-squad2",
    # "model_name": "google/rembert",
    "model_class": "bare", # bare, qa
    "max_len": 384,
    "doc_stride": 128,
    "dropout": 0.0,
    "init_weights": True,
    "init_layers": 1,
    # "freeze_layers": 0,
    "datasets": [
        "mlqa:v1",
        "xquad:v1",
        "squad_translated_tamil:v1",
    ],
    "models": [
        "deepset-xlm-roberta-large-squad2:v1",
    ],
    "runs": [
        "1cbdwt4m",
    ],
}


In [12]:
if Config.debug:
    config_defaults["n_fold"] = 3
    config_defaults["epochs"] = 1
    Config.print_freq = 10


In [13]:
class Struct:
    def __init__(self, entries):
        self.__dict__.update(**entries)


In [14]:
config = Struct(config_defaults)


# Load Artifacts

In [15]:
api = wandb.Api()

for n, run_id in enumerate(config.runs):
    if not os.path.exists(run_id):
        os.makedirs(run_id)

    run_path = f"{Config.wandb_entity}/{Config.wandb_project}/{run_id}"
    run = api.run(run_path)

    try:
        run.file("oof_df.csv").download(run_id)
    except wandb.CommError:
        # Already downloaded.
        pass

    oof = pd.read_csv(f"{run_id}/oof_df.csv")
    break


In [16]:
oof

,id,context,question,answer_text,answer_start,language,answers,fold,prediction,jaccard
0,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358.0,tamil,"{'answer_start': [2358], 'text': ['காசுமீரில்']}",0,காசுமீரில்,1.000000
1,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68.0,tamil,"{'answer_start': [68], 'text': ['தாலாட்டு']}",0,தாலாட்டு,1.000000
2,d419db018,மின்னணுவியல் (Electronics) மின்னணுக்கள் அல்லது...,திரிதடையங்களைப் பயன்படுத்திய முதல் நிறுவனம் எது?,IBM,4171.0,tamil,"{'answer_start': [4171], 'text': ['IBM']}",0,IBM,1.000000
3,2a5ba78e2,முதல் உலகப்போர் என்பது உலகம் தழுவிய அளவில் இடம...,முதலாம் உலகப்போர் எப்பொழுது துவங்கியது?,1914ம்,1070.0,tamil,"{'answer_start': [1070], 'text': ['1914ம்']}",0,1914,0.000000
4,b8ac8836e,பாரிஸ் அல்லது பாரி எனப்படுவது பிரான்ஸ் நாட்டின...,பாரிஸ் எந்த நாட்டின் தலைநகரம் ஆகும்?,பிரான்ஸ்,30.0,tamil,"{'answer_start': [30], 'text': ['பிரான்ஸ்']}",0,பிரான்ஸ்,1.000000
...,...,...,...,...,...,...,...,...,...,...
11285,10163,யேல் வளாகத்திற்கு அருகில் உள்ள ஒரு தேசிய வரலாற...,எந்த குறிப்பிடத்தக்க புதிய புகலிடமாக குடியிருப...,எலி விட்னி,74.0,tamil,"{'answer_start': [74.0], 'text': ['எலி விட்னி']}",4,ஜார்ஜ் வாஷிங்டன்,0.000000
11286,10169,"1996 முதல் 2002 வரை, டுவாலு சிறந்த முறையில் பச...",செலவில் என்ன உயர்வு குவளலின் உள்நாட்டு வளர்ச்ச...,எரிபொருள் மற்றும் உணவு,370.0,tamil,"{'answer_start': [370.0], 'text': ['எரிபொருள் ...",4,எரிபொருள்,0.333333
11287,10172,எமிட் அரசாங்கத்தின் பணி மற்றும் நோக்கங்களை ஆதர...,மாணவர்களின் எண்ணிக்கை என்ன?,50,421.0,tamil,"{'answer_start': [421.0], 'text': ['50']}",4,50,1.000000
11288,10173,"தஜிகிஸ்தான்(I /tɑːdʒiːkᵻstɑːn/, /tədʒiːkᵻstæn/...",தஜிகிஸ்தானில் எத்தனை பேர் கணக்கிடப்படுகிறார்கள்?,8 மில்லியன்,353.0,tamil,"{'answer_start': [353.0], 'text': ['8 மில்லியன...",4,8 மில்லியன்,1.000000


# Tokenize

In [17]:
tokenizer = T.AutoTokenizer.from_pretrained("deepset/xlm-roberta-base-squad2")


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/605 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [61]:
if True:
    context_tokens = [tokenizer(txt)["input_ids"] for txt in oof["context"]]
    question_tokens = [tokenizer(txt)["input_ids"] for txt in oof["question"]]
    answer_tokens = [tokenizer(txt)["input_ids"] for txt in oof["answer_text"]]
    prediction_tokens = [tokenizer(txt)["input_ids"] for txt in oof["prediction"]]

    oof["num_tokens_context"] = [len(tok) for tok in context_tokens]
    oof["num_chars_context"] = [len(tok) for tok in oof["context"]]
    oof["num_tokens_question"] = [len(tok) for tok in question_tokens]
    oof["num_chars_question"] = [len(tok) for tok in oof["question"]]
    oof["num_tokens_answer"] = [len(tok) for tok in answer_tokens]
    oof["num_chars_answer"] = [len(tok) for tok in oof["answer_text"]]
    oof["num_tokens_prediction"] = [len(tok) for tok in prediction_tokens]
    oof["num_chars_prediction"] = [len(tok) for tok in oof["prediction"]]

    oof.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11290 entries, 0 to 11289
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     11290 non-null  object 
 1   context                11290 non-null  object 
 2   question               11290 non-null  object 
 3   answer_text            11290 non-null  object 
 4   answer_start           11290 non-null  float64
 5   language               11290 non-null  object 
 6   answers                11290 non-null  object 
 7   fold                   11290 non-null  int64  
 8   prediction             11290 non-null  object 
 9   jaccard                11290 non-null  float64
 10  num_tokens_context     11290 non-null  int64  
 11  num_chars_context      11290 non-null  int64  
 12  num_tokens_question    11290 non-null  int64  
 13  num_chars_question     11290 non-null  int64  
 14  num_tokens_answer      11290 non-null  int64  
 15  nu

In [21]:
oof

,id,context,question,answer_text,answer_start,language,answers,fold,prediction,jaccard,num_tokens_context,num_chars_context,num_tokens_question,num_chars_question,num_tokens_answer,num_chars_answer,num_tokens_prediction,num_chars_prediction
0,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358.0,tamil,"{'answer_start': [2358], 'text': ['காசுமீரில்']}",0,காசுமீரில்,1.000000,2316,7647,11,26,6,10,6,10
1,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68.0,tamil,"{'answer_start': [68], 'text': ['தாலாட்டு']}",0,தாலாட்டு,1.000000,2715,8493,20,75,5,8,5,8
2,d419db018,மின்னணுவியல் (Electronics) மின்னணுக்கள் அல்லது...,திரிதடையங்களைப் பயன்படுத்திய முதல் நிறுவனம் எது?,IBM,4171.0,tamil,"{'answer_start': [4171], 'text': ['IBM']}",0,IBM,1.000000,2572,8796,14,48,3,3,3,3
3,2a5ba78e2,முதல் உலகப்போர் என்பது உலகம் தழுவிய அளவில் இடம...,முதலாம் உலகப்போர் எப்பொழுது துவங்கியது?,1914ம்,1070.0,tamil,"{'answer_start': [1070], 'text': ['1914ம்']}",0,1914,0.000000,9866,33997,14,39,4,6,3,4
4,b8ac8836e,பாரிஸ் அல்லது பாரி எனப்படுவது பிரான்ஸ் நாட்டின...,பாரிஸ் எந்த நாட்டின் தலைநகரம் ஆகும்?,பிரான்ஸ்,30.0,tamil,"{'answer_start': [30], 'text': ['பிரான்ஸ்']}",0,பிரான்ஸ்,1.000000,2895,9861,13,36,4,8,4,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11285,10163,யேல் வளாகத்திற்கு அருகில் உள்ள ஒரு தேசிய வரலாற...,எந்த குறிப்பிடத்தக்க புதிய புகலிடமாக குடியிருப...,எலி விட்னி,74.0,tamil,"{'answer_start': [74.0], 'text': ['எலி விட்னி']}",4,ஜார்ஜ் வாஷிங்டன்,0.000000,221,697,45,144,7,10,9,16
11286,10169,"1996 முதல் 2002 வரை, டுவாலு சிறந்த முறையில் பச...",செலவில் என்ன உயர்வு குவளலின் உள்நாட்டு வளர்ச்ச...,எரிபொருள் மற்றும் உணவு,370.0,tamil,"{'answer_start': [370.0], 'text': ['எரிபொருள் ...",4,எரிபொருள்,0.333333,383,1450,17,64,6,22,4,9
11287,10172,எமிட் அரசாங்கத்தின் பணி மற்றும் நோக்கங்களை ஆதர...,மாணவர்களின் எண்ணிக்கை என்ன?,50,421.0,tamil,"{'answer_start': [421.0], 'text': ['50']}",4,50,1.000000,192,763,7,27,3,2,3,2
11288,10173,"தஜிகிஸ்தான்(I /tɑːdʒiːkᵻstɑːn/, /tədʒiːkᵻstæn/...",தஜிகிஸ்தானில் எத்தனை பேர் கணக்கிடப்படுகிறார்கள்?,8 மில்லியன்,353.0,tamil,"{'answer_start': [353.0], 'text': ['8 மில்லியன...",4,8 மில்லியன்,1.000000,288,732,16,48,7,11,7,11


# Checking for unknown tokens

In [22]:
if False:
    context_tokens_flat = sum(context_tokens, [])
    question_tokens_flat = sum(question_tokens, [])
    answer_tokens_flat =  sum(answer_tokens, [])
    prediction_tokens_flat =  sum(prediction_tokens, [])

    # context 以外には未知の token はない
    unk = tokenizer.unk_token_id

    unk in context_tokens_flat, unk in question_tokens_flat, unk in answer_tokens_flat, unk in prediction_tokens_flat

    # context に含まれる未知の token の数と割合 -> 無視できるくらい低い？
    num_unk_tokens = sum([tok == unk for tok in context_tokens_flat])
    num_unk_tokens, num_unk_tokens/len(context_tokens_flat)

KeyboardInterrupt: ignored

# Same with Muril

In [ ]:
if False:
    tokenizer_muril = T.AutoTokenizer.from_pretrained("google/muril-base-cased")

    context_tokens_muril = [tokenizer_muril(txt)["input_ids"] for txt in oof["context"]]
    question_tokens_muril = [tokenizer_muril(txt)["input_ids"] for txt in oof["question"]]
    answer_tokens_muril = [tokenizer_muril(txt)["input_ids"] for txt in oof["answer_text"]]
    prediction_tokens_muril = [tokenizer_muril(txt)["input_ids"] for txt in oof["prediction"]]

    context_tokens_flat_muril = sum(context_tokens_muril, [])
    question_tokens_flat_muril = sum(question_tokens_muril, [])
    answer_tokens_flat_muril =  sum(answer_tokens_muril, [])
    prediction_tokens_flat_muril =  sum(prediction_tokens_muril, [])

    unk_muril = tokenizer_muril.unk_token_id

    print("Unk token in context, question, answer, prediction")
    print(unk_muril in context_tokens_flat_muril, unk_muril in question_tokens_flat_muril, unk_muril in answer_tokens_flat_muril, unk_muril in prediction_tokens_flat_muril)

    print("Num unk tokens in context, question, answer, prediction")
    sum([tok == unk_muril for tok in context_tokens_flat_muril]), sum([tok == unk_muril for tok in question_tokens_flat_muril]), sum([tok == unk_muril for tok in answer_tokens_flat_muril]), sum([tok == unk_muril for tok in prediction_tokens_flat_muril])


Unk token in context, question, answer, prediction
True True True True
Num unk tokens in context, question, answer, prediction


(1403, 19, 2, 1)

# Same with RemBERT

In [ ]:
if False:
    tokenizer_rembert = T.AutoTokenizer.from_pretrained("google/rembert")

    context_tokens_rembert = [tokenizer_rembert(txt)["input_ids"] for txt in oof["context"]]
    question_tokens_rembert = [tokenizer_rembert(txt)["input_ids"] for txt in oof["question"]]
    answer_tokens_rembert = [tokenizer_rembert(txt)["input_ids"] for txt in oof["answer_text"]]
    prediction_tokens_rembert = [tokenizer_rembert(txt)["input_ids"] for txt in oof["prediction"]]

    context_tokens_flat_rembert = sum(context_tokens_rembert, [])
    question_tokens_flat_rembert = sum(question_tokens_rembert, [])
    answer_tokens_flat_rembert =  sum(answer_tokens_rembert, [])
    prediction_tokens_flat_rembert =  sum(prediction_tokens_rembert, [])

    unk_rembert = tokenizer_rembert.unk_token_id

    print("Unk token in context, question, answer, prediction")
    print(unk_rembert in context_tokens_flat_rembert, unk_rembert in question_tokens_flat_rembert, unk_rembert in answer_tokens_flat_rembert, unk_rembert in prediction_tokens_flat_rembert)

    print("Num unk tokens in context, question, answer, prediction")
    sum([tok == unk_rembert for tok in context_tokens_flat_rembert]), sum([tok == unk_rembert for tok in question_tokens_flat_rembert]), sum([tok == unk_rembert for tok in answer_tokens_flat_rembert]), sum([tok == unk_rembert for tok in prediction_tokens_flat_rembert])

Downloading:   0%|          | 0.00/263 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3137 > 256). Running this sequence through the model will result in indexing errors


Unk token in context, question, answer, prediction
True False False False
Num unk tokens in context, question, answer, prediction


(119, 0, 0, 0)

# Looking at char level

In [23]:
contexts = oof["context"]
answers = oof["answer_text"]
predictions = oof["prediction"]

all_chars_ctx = "".join(contexts)
all_chars_ans = "".join(answers)
all_chars_preds = "".join(predictions)

unq_chars_ctx = sorted(list(set(all_chars_ctx)))
unq_chars_ans = sorted(list(set(all_chars_ans)))
unq_chars_preds = sorted(list(set(all_chars_preds)))


In [24]:
print("Contexts: ", len(contexts), contexts.nunique())
print("Answers: ", len(answers), answers.nunique())
print("Predictions: ", len(predictions), predictions.nunique())


Contexts:  11290 8948
Answers:  11290 9537
Predictions:  11290 9411


In [25]:
# 含まれている文字の一覧
"".join(unq_chars_ctx)


'\t\n !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\xa0¢£¥§¨«®¯°±²³´µ·º»¼½¾¿ÀÁÇÉÍÎÖ×ØÚÜßàáâãäåæçèéêëìíîïðñòóôõö÷øùúüýþĀāăąćčďĐđēęěğĩīİıķŁłńņňŋŌōőœřŚśşŠšťūųźżžƏƒơưǎǐǔǚɐɑɒɔɕɖəɛɜɟɡɣɦɨɪɫɯɲɳɴɵɹɽɾʀʁʂʃʈʊʋʌʏʑʒʔʕʝʰʲʻʿˀˈˌː˙˚ˢ̞̥̩̪̯̰̀́̃̆̊̚͡ͰͱͲͳͶͷΐΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩάέήίαβγδεζηθικλμνξοπρςστυφχψωόύώϘϙϚϛϜϝϞϟϠϡϷϸϺϻЈАБВГДЖЗИКЛМНОПРСТУФХЧШЭЯабвгдежзийклмнопрстуфхцчшщыьюяёіјћўғҙҳҶҷҺһӘәԵԻՀՄագեզըթիլծկհղմյնշոջտրցւքևְִֵֶַָֹּׁׂאבגדהוזחיכלםמןנספקרשתءآأؤإئابةتثجحخدذرزسشصضطظعغفقكلمنهوىيًَُِّْٰٹټپچډښکګگھہۂۃیەܒܠऀँंःऄअआइईउऊऋऌऍऎएऐऑऒओऔकखगघङचछजझञटठडढणतथदधनऩपफबभमयरऱलळऴवशषसहऺऻ़ऽािीुूृॄॅॆेैॉॊोौ्ॎॏॐ॒॑॓॔ॕॖॗक़ख़ग़ज़ड़ढ़फ़य़ॠॡॢॣ।॥०१२३४५६७८९॰ॱॲॳॴॵॶॷॸॹॺॻॼॽॾॿঁংঅআউএওকগঙচছজঞটঠডঢণতথদধনপবভমযরলশষসহ়ািীুৃেোৌ্ৎ০১২৬ৰৱਊਕਗਜਥਦਨਮਰਲਸਹਾਿੀੁੂ੍ੱંઈટણદનપબભમરલળવસાિીુેો્ஂஃஅஆஇஈஉஊஎஏஐஒஓஔகஙசஜஞடணதநனபமயரறலளழவஷஸஹாிீுூெேைொோௌ்ௗ௦௧௭௯ంకగచజటడదనబమయరవశషసహాిీుూెేైో్ಂಅಕಗಟಡದನಪಬರಳಶಸಾಿುೂೆೇ್ംഅകഗതദനപബമയരലവശസാിീുൃൊ്යලාกขคงญดตทนบปพมยรฤวสหอะัาำิีืุ฿เใ่้๋์ກດບອັີ་།གདནབརལསེུྩྭကခဂငစဋတထဒနပဖဘမယရလသအာိီုူေဲ့း္်ျြွဿባዓይកគងជតទនពមយរវសអឧា

In [26]:
"".join(unq_chars_ans)


' "#$%&\'()*+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz~¢£°±²½¿×Øàâéöüćźżǎʰβπφँंःअआइईउऊऋऍएऐऑओऔकखगघचछजझञटठडढणतथदधनपफबभमयरऱलवशषसह़ािीुूृॅेैॉोौ्क़ख़ज़ड़ढ़फ़ॠ।०१२३४५६७८९॰ஃஅஆஇஈஉஊஎஏஐஒஓகஙசஜஞடணதநனபமயரறலளழவஷஸஹாிீுூெேைொோௌ்௭ကခငတနဘာုေး်\u200b\u200d–‘’“”′⁄€₹−≥⊆黑'

In [27]:
"".join(unq_chars_preds)


'\n !"#$%&\'()+,-./0123456789:;=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz|~¢£§°±²½¿×Øàâéíïóöüāżǎʰβπφазкмоँंःअआइईउऊऋऍएऐऑओऔकखगघचछजझञटठडढणतथदधनपफबभमयरलवशषसह़ािीुूृॅेैॉोौ्क़ख़ज़ड़ढ़फ़ॠ।०१२३४५६७८९॰ஃஅஆஇஈஉஊஎஏஐஒஓஔகஙசஜஞடணதநனபமயரறலளழவஷஸஹாிீுூெேைொோௌ்ௗขยวีเ\u200b\u200d–’“”′⁄€₹−≥⊆\ufeff'

## prediction にだけ含まれる文字の確認

In [28]:
set(all_chars_preds) - set(all_chars_ans)

{'\n',
 '!',
 '>',
 '|',
 '§',
 'í',
 'ï',
 'ó',
 'ā',
 'а',
 'з',
 'к',
 'м',
 'о',
 'ஔ',
 'ௗ',
 'ข',
 'ย',
 'ว',
 'ี',
 'เ',
 '\ufeff'}

In [29]:
set(all_chars_ans) - set(all_chars_preds)

{'*',
 'ć',
 'ź',
 'ऱ',
 '௭',
 'က',
 'ခ',
 'င',
 'တ',
 'န',
 'ဘ',
 'ာ',
 'ု',
 'ေ',
 'း',
 '်',
 '‘',
 '黑'}

In [36]:
# 先頭と末尾の改行は除外して良さそう
# 真ん中に入っている 改行をどうするか。。

oof[oof["prediction"].str.contains("\n")][["id", "answer_text", "prediction"]]

,id,context,question,answer_text,answer_start,language,answers,fold,prediction,jaccard,num_tokens_context,num_chars_context,num_tokens_question,num_chars_question,num_tokens_answer,num_chars_answer,num_tokens_prediction,num_chars_prediction
2387,e6215a423,"चीनी साहित्य अपनी प्राचीनता, विविधता और ऐतिहास...",चीन के सर्वप्रथम जनकवि किसे माना जाता हैं?,चू य्वान्,799.0,hindi,"{'answer_start': [799], 'text': ['चू य्वान्']}",1,चू य्वान \nचू य्वान्,0.666667,6374,21654,15,42,6,9,10,19


In [53]:
# 前後にあるBOMも除外する

oof[oof["prediction"].str.contains('\ufeff')][["id", "answer_text", "prediction"]]

,id,answer_text,prediction
5988,6208,एर्गनज़नशूलेन,﻿एर्गनज़नशूलेन
6032,6438,अभाज्य,﻿अभाज्य


In [65]:
oof.loc[(oof["answer_text"].str.endswith("ம்")) & (oof["jaccard"] != 1.0), :]

,id,context,question,answer_text,answer_start,language,answers,fold,prediction,jaccard,num_tokens_context,num_chars_context,num_tokens_question,num_chars_question,num_tokens_answer,num_chars_answer,num_tokens_prediction,num_chars_prediction
3,2a5ba78e2,முதல் உலகப்போர் என்பது உலகம் தழுவிய அளவில் இடம...,முதலாம் உலகப்போர் எப்பொழுது துவங்கியது?,1914ம்,1070.0,tamil,"{'answer_start': [1070], 'text': ['1914ம்']}",0,1914,0.000000,9866,33997,14,39,4,6,3,4
18,d75178d00,கதிரவ அமைப்பு (Solar System) அல்லது சூரியக் கு...,நமது சூரிய மண்டலத்தில் எத்தனை கிரகங்கள் உள்ளன?,"எட்டு கோள்களையும், ஐந்து குறுங்கோள்களையும்",196.0,tamil,"{'answer_start': [196], 'text': ['எட்டு கோள்கள...",0,எட்டு,0.250000,3698,12462,15,46,14,42,4,5
22,80452bc4d,கால்வாய் எனப்படுவது நீர்ப்பாசனத்துக்காக கால்வா...,உலகின் மிகப்பெரிய மனிதனால் உருவாக்கப்பட்ட கால்...,பெரும்,1928.0,tamil,"{'answer_start': [1928], 'text': ['பெரும்']}",0,சீனாவின் பெரும் கால்வாய்,0.333333,2778,10260,16,55,3,6,7,24
45,363dddf6f,தமிழ்நாடு (Tamil Nadu) என்து இந்தியாவின் 29 ம...,தமிழ்நாட்டின் பழைய பெயர் என்ன?,சென்னை மாகாணம்,783.0,tamil,"{'answer_start': [783], 'text': ['சென்னை மாகாண...",0,மதராசு ஸ்டேட்,0.000000,7475,27171,9,30,5,14,8,13
54,ec2871c3f,சிங்கம் என்பது பாலூட்டி வகையைச் சேர்ந்த ஒரு கா...,சிங்கத்தின் சராசரி ஆயுட்காலம் என்ன?,ஆண் சிங்கங்களின் சராசரி ஆயுட்காலம் 12 வருடங்கள...,1914.0,tamil,"{'answer_start': [1914], 'text': ['ஆண் சிங்கங்...",0,12,0.111111,1513,5575,15,35,33,106,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11167,9545,1698 ஆம் ஆண்டில் வழங்கப்பட்ட கிழக்கு இந்தியா ந...,EIC கைகளில் அவரது முன்னோடிகளில் லயன் வைத்திருந...,கிரீடம்,266.0,tamil,"{'answer_start': [266.0], 'text': ['கிரீடம்']}",4,ஒரு குறுக்கு குலுக்கல்,0.000000,168,500,17,56,5,7,8,22
11188,9646,ஒரு நிலையான சொற்பொருள் (ஒரு தரப்படுத்தப்பட்ட ச...,ஸ்டாண்டர்ட் பிலிப்பைன் ஆங்கிலம் என்பது என்ன மொ...,ஆங்கிலம்,617.0,tamil,"{'answer_start': [617.0], 'text': ['ஆங்கிலம்']}",4,ஆங்கில,0.000000,228,855,23,76,4,8,3,6
11199,9713,"சமஸ்கிருதம் ஒரு வாய்வழி சமுதாயத்தில் உருவானது,...",எந்த வகை பாரம்பரியத்தின் கீழ் சமஸ்கிருதம் தொடங...,வாய்வழி பாரம்பரியம்,106.0,tamil,"{'answer_start': [106.0], 'text': ['வாய்வழி பா...",4,வாய்வழி,0.500000,146,524,18,53,9,19,5,7
11207,9745,இந்த வயதில் கிரேக்கர்கள் பெரிய நகரங்களை நோக்கி...,மேற்கோள்கள் தங்கள் தொடக்கத்தை தங்களை நினைவுபடு...,"எவ்வாறாயினும், கிரேக்கர்கள் தங்கள் கடந்த காலத்...",183.0,tamil,"{'answer_start': [183.0], 'text': ['எவ்வாறாயின...",4,கிரேக்கர்கள்,0.066667,189,680,16,58,37,147,6,12


In [66]:
oof[oof["prediction"].str.contains('\)')][["answer_text", "prediction"]]

,answer_text,prediction
359,रियर इंजन,"130 ब्रेक अश्वशक्ति (97 कि॰वाट), 1,289 घन सेंट..."
409,"यह गलत या अपूर्ण जानकारी, सिद्धांत, मूर्खता, आ...",एक ऐसा विश्वास जो एक रोगात्मक (किसी बीमारी या ...
475,"36.6 मी॰ (120.08 फीट) लंबा, 18 मी॰ (59.06 फीट)...",13.5 मी॰ (44.29 फीट) ऊँचा
608,"अलाइड-सिग्नल (AlliedSignal), एलीसन इंजिन (Alli...","अलाइड-सिग्नल (AlliedSignal), एलीसन इंजिन (Alli..."
687,1900,19 अप्रैल (नई शैली में 2 मई) 1900
...,...,...
10845,முற்போக்கான பழமைவாத (பிசி) மூலோபாயவாதிகள்,பிசி) மூலோபாயவாதிகள்
11007,கிழக்கு ஹான்,"65 கி.மீ.லியு ying (d. 71 விளம்பரம்), ஹான் (ஆர..."
11027,1.6 கி.மீ,1.6 கி.மீ. (1.0 மைல்) தடிமனாக இருக்கும்.கண்டத்...
11029,கல்வி (கட்டுப்பாட்டு அமைப்புகள்) மற்றும் விமான...,கல்வி (கட்டுப்பாட்டு அமைப்புகள்) மற்றும் விமானம்


In [ ]:
oof[oof["prediction"].str.contains('ढ')][["answer_text", "prediction"]]

,answer_text,prediction
1006,प्रोटोज़ोआ,संक्रमित रक्त को चढ़ाने से


## Ground truth に含まれる文字を確認

In [ ]:
oof[oof["answer_text"].str.contains("கி.பி")][["answer_text", "prediction"]]

,answer_text,prediction
669,கி.பி.1510,\nகி.பி.1510


In [ ]:
oof[oof["answer_text"].str.contains("கி.மு")][["answer_text", "prediction"]]

,answer_text,prediction
250,கி.மு 3000,கி.மு 6000
260,கி.மு 1400-1000,1839
670,கி.மு. ஐந்தாம் நூற்றாண்டில்,கி.மு 470/469


In [ ]:
oof[oof["answer_text"].str.contains("கி.மீ")][["answer_text", "prediction"]]

,answer_text,prediction
58,"10,180,000 ச.கி;மீகள்","10,180,000 ச.கி;மீகள்"
267,6400 கி.மீ.கள்,6400 கி.மீ.கள்
688,6400 கி.மீ.கள்,6400 கி.மீ.கள்
701,6650 கி.மீ,6650 கி.மீ
721,147.99 கி.மீ,147.99 கி.மீ.2
728,147.99 கி.மீ.,147.99 கி.மீ.2
922,174 கி.மீ²,174 கி.மீ².


In [ ]:
oof[oof["answer_text"].str.contains("ई")][["answer_text", "prediction"]]

,answer_text,prediction
81,ईक्वस कैबेलस,ईक्वस
121,चग़ताई,चग़ताई भाषा
128,२७ मई १९५७,२७ मई १९५७
135,थाई,सेंट्रल थाई
174,वाई एस आर कांग्रेस पार्टी,वाई एस आर कांग्रेस पार्टी
185,563 ईसा पूर्व,563 ईसा पूर्व
195,लातवियाई भाषा,लातवियाई
211,328 ई.,328 ई.
301,ईसाई धर्म,ईसाई
303,अप्रैल से जुलाई,अप्रैल से जुलाई तक


In [ ]:
oof[oof["answer_text"].str.contains("ई.पू")][["answer_text", "prediction"]]

,answer_text,prediction
413,331 ई.पू.,331 ई.पू.
604,27 ई.पू.,(27 ई.पू.
854,287 ई.पू.,287 ई.पू.
1090,400 ई॰पू॰ 100 ई॰ सन् के बीच,400 ई॰पू॰ 100 ई॰ सन्
